In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import seaborn as sns
import re
#import commons as cmns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import PowerTransformer


In [12]:
data=pd.read_csv('../data/btc_data_lastest_10-21.csv',sep=',')
data = data.drop(["Unnamed: 0"], axis=1)

interval1 = (data['Date'] >= '2013/04/01') & (data['Date'] <= '2016/04/01')
interval2=(data['Date'] >= '2013/04/01') & (data['Date'] <= '2017/04/01')
interval3=(data['Date'] >= '2010/04/01') & (data['Date'] <= '2021/12/31')

one=data['priceUSD'].shift(-1,fill_value=1)
seven=data['priceUSD'].shift(-7,fill_value=1)
fourteen=data['priceUSD'].shift(-14,fill_value=1)
thirty=data['priceUSD'].shift(-30,fill_value=1)
sixty=data['priceUSD'].shift(-60,fill_value=1)
ninety=data['priceUSD'].shift(-90,fill_value=1)

df=data.loc[interval3]

X=df.iloc[:,2:]
y=df.iloc[:,1:2]
#X = X.drop(["priceUSD"], axis=1)

y=np.ravel(y)

min_max_scaler = MinMaxScaler()
#X = min_max_scaler.fit_transform(X)



In [13]:
X_train, X_test, y_train, y_test = train_test_split(min_max_scaler.fit_transform(X), y, test_size=0.25, random_state=0)


In [22]:

forest = RandomForestRegressor(
   n_jobs = -1, 
   n_estimators = 1000
)

# Train the classifier
forest.fit(min_max_scaler.fit_transform(X), y)


#sfm = SelectFromModel(forest, threshold=0.8)

# Train the selector
#sfm.fit(min_max_scaler.fit_transform(X), y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [23]:
# Print the name and gini importance of each feature
for feature in zip(X.columns, forest.feature_importances_):
    print(feature)

('transactions', 6.693667168234609e-07)
('size', 6.570658497685955e-07)
('sentbyaddress', 1.0835187803184465e-06)
('difficulty', 7.701483776239039e-06)
('hashrate', 5.095022631200769e-07)
('mining_profitability', 4.627384352704417e-08)
('sentinusdUSD', 2.5852427087817574e-06)
('transactionfeesUSD', 5.793533652637996e-07)
('median_transaction_feeUSD', 1.998621135292852e-06)
('confirmationtime', 8.20370120787331e-07)
('transactionvalueUSD', 2.138961488143281e-06)
('mediantransactionvalueUSD', 6.702756838980914e-07)
('activeaddresses', 6.048743634092075e-07)
('top100cap', 7.597059923675344e-07)
('fee_to_rewardUSD', 3.348975625969589e-07)
('transactions3sma', 8.088585968356368e-07)
('transactions7sma', 2.712400939006862e-07)
('transactions14sma', 4.854861127869772e-07)
('transactions30sma', 2.2672993339150584e-07)
('transactions90sma', 3.243929318708372e-07)
('transactions3ema', 6.828811841283747e-07)
('transactions7ema', 7.423855400729742e-07)
('transactions14ema', 3.20586483498993e-07)
(

In [26]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

# define number of features to evaluate
num_features = [i for i in range(10, 20)]

# enumerate each number of features
results = list()
for k in num_features:
    # create pipeline
    model = RandomForestRegressor(random_state=7,n_jobs=-1)
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    pipeline = Pipeline(steps=[('sel',fs), ('lr', model)])
    # evaluate the model
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    results.append(scores)
    # summarize the results
    print('>%d %.3f (%.3f)' % (k, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=num_features, showmeans=True)
pyplot.show()

KeyboardInterrupt: 

In [31]:
# for feature_list_index in sfm.get_support(indices=True):
#     print(X.columns[feature_list_index])
# sfm.get_support(indices=True)

features_list=[]
technical_indicators=['sma','ema','wma','trx','mom','std','var','rsi','roc']
periods=['3','7','14','30','90']
for i in technical_indicators:
    for j in periods:
        try:
            string=str(j)+str(i)
            X=df.filter(like=string,axis=1)
            X=SimpleImputer(missing_values=0,strategy='most_frequent').fit_transform(X)
            X=pd.DataFrame(X)
            X.columns=df.filter(like=string,axis=1).columns
            rf1=RandomForestRegressor(random_state=7,n_jobs=-1)
            rfecv=RFECV(rf1,step=0.9,min_features_to_select=1,verbose=1,cv=5,scoring='neg_mean_absolute_error', n_jobs=-1)
            rfecv.fit(X,y)
            if rfecv.n_features_>1:
                rf1.fit(X,y)
                maximp=rf1.feature_importances_.max()
                for x in range(len(rf1.feature_importances_)):
                    if maximp==rf1.feature_importances_[x]:
                        new_features = X.columns[x]
            else:
                mask = rfecv.get_support()
                new_features = X.columns[mask]
            features_list.append(str(new_features))
            print(string+': '+new_features)
        except:
            pass

Fitting estimator with 16 features.


/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 2 features.


/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Index(['3sma: price3smaUSD'], dtype='object')
Fitting estimator with 16 features.


...

Index(['14roc: difficulty14roc'], dtype='object')
Fitting estimator with 16 features.


/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", Fut

30roc: size30roc
Fitting estimator with 16 features.


/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/.pyenv/versions/3.7.9/envs/nlp/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", Fut

90roc: top100cap90roc


In [33]:
features_list
l1=[]
for j in range(len(features_list)):
    result1 = re.search("'(.*)'],", features_list[j])
    if result1!=None:
        l1.append(result1.group(1))
for i in range(len(features_list)):
    result2 = re.search('.*',features_list[i])
    if len(result2.group(0))<33:
        l1.append(result2.group(0))

In [43]:
X_new = min_max_scaler.fit_transform(df[l1])

In [47]:
# Using boruta and rfr

from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np


boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 10,
    verbose = True
)

boruta.fit(X_new, y)

green_area = df[l1].columns[boruta.support_].to_list()
blue_area = df[l1].columns[boruta.support_weak_].to_list()

print('Features with high Support:', green_area)
print('Features with low Support:', blue_area)

Iteration: 1 / 10
Iteration: 2 / 10
Iteration: 3 / 10
Iteration: 4 / 10
Iteration: 5 / 10
Iteration: 6 / 10
Iteration: 7 / 10
Iteration: 8 / 10
Iteration: 9 / 10


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	17
Tentative: 	3
Rejected: 	16
Features with high Support: ['price3smaUSD', 'price7smaUSD', 'price14smaUSD', 'price30smaUSD', 'price3emaUSD', 'price7emaUSD', 'price14emaUSD', 'price30emaUSD', 'price90emaUSD', 'price3wmaUSD', 'price7wmaUSD', 'price14wmaUSD', 'price30wmaUSD', 'price90wmaUSD', 'transactionvalue90smaUSD', 'price3momUSD', 'price90stdUSD']
Features with low Support: ['top100cap30trx', 'price90momUSD', 'top100cap90rsi']


In [48]:
df_with_features = df[['Date', 'priceUSD']+green_area]
df_with_features.to_csv('../data/new/features_regression_boruta_plus_rf.csv', index=False)

In [52]:
!pip install bs4
!pip install pytrends
!pip install cryptory

You should consider upgrading via the '/root/.pyenv/versions/3.7.9/envs/nlp/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 6.3 MB 13.1 MB/s eta 0:00:01
You should consider upgrading via the '/root/.pyenv/versions/3.7.9/envs/nlp/bin/python3.7 -m pip install --upgrade pip' command.
  Using cached cryptory-0.1.1.tar.gz (11 kB)
  Created wheel for cryptory: filename=cryptory-0.1.1-py3-none-any.whl size=9190 sha256=ae0202bc22a450c16157b4775259e3a255c3c888fe541c4b23f7a01f66f59a91
  Stored in directory: /root/.cache/pip/wheels/a6/2d/d0/226c24d2fc279c557cb97ec0a2587c6750650f10098eaae9fc
Successfully built cryptory
You should consider upgrading via the '/root/.pyenv/versions/3.7.9/envs/nlp/bin/python3.7 -m pip install --upgrade pip' command.


In [54]:
!pip install html5lib

  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
You should consider upgrading via the '/root/.pyenv/versions/3.7.9/envs/nlp/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
from cryptory import Cryptory

# initialise object 
# pull data from start of 2017 to present day
my_cryptory = Cryptory(from_date = "2017-01-01")

# get historical bitcoin prices from coinmarketcap
my_cryptory.extract_coinmarketcap("btc")

HTTPError: HTTP Error 404: Not Found

In [ ]:
my_cryptory.extract_coinmarketcap("bitcoin")

In [6]:
!pip install cryptocmd

     |████████████████████████████████| 47 kB 5.6 MB/s  eta 0:00:01
You should consider upgrading via the '/root/.pyenv/versions/3.7.9/envs/nlp/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
from cryptocmd import CmcScraper

# initialise scraper without time interval
scraper = CmcScraper("btc")

# get raw data as list of list
headers, data = scraper.get_data()

# get data in a json format
xrp_json_data = scraper.get_data("json")

# export the data as csv file, you can also pass optional `name` parameter
scraper.export("csv", name="xrp_all_time")

# Pandas dataFrame for the same data
df = scraper.get_dataframe()

In [6]:
df['Date']

0      2021-08-16
1      2021-08-15
2      2021-08-14
3      2021-08-13
4      2021-08-12
          ...    
3028   2013-05-02
3029   2013-05-01
3030   2013-04-30
3031   2013-04-29
3032   2013-04-28
Name: Date, Length: 3033, dtype: datetime64[ns]